We investigate two different heterologous pathways for production of progesterone in S. cerevisiae. The heterologous pathways are added to the chosen S. cerevisiae GSM. 
iMM model.

In [1]:
from cobra.io import read_sbml_model

model = read_sbml_model('models/iMM904.xml')
model

Name,iMM904
Memory address,0x0201e6d95e80
Number of metabolites,1226
Number of reactions,1577
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


Implementation of pathway 1

In [5]:
from cobra import Model, Reaction, Metabolite

# 5 metabolites
zymosterol = model.metabolites.get_by_id('M_zymst_c') # found in the model
zymosterol.id = "zymosterol"

dehydrocholesterol = Metabolite(
    '7-dehydrocholersterol', 
    formular = 'C27H44O',
    name = 'cholesterol-precursor',
    compartment = 'c')

cholesterol = Metabolite(
    'cholesterol',
    formular = 'C27H46O',
    name = 'membrane-structure-steroid',
    compartment = 'c')

pregnenolone = Metabolite(
    'pregnenolone',
    formula = 'C21H32O2',
    name = 'steroid-hormone-precursor',
    compartment = 'c')

# 4 reactions
dehydrocholesterol_reductase_24 = Reaction('DHCR24')
dehydrocholesterol_reductase_7 = Reaction('DHCR7')
cholesterol_side_chain_cleavage_enzyme = Reaction('CYP11A1')
hydroxysteroid_dehydrogenase_3_beta = Reaction('3beta-HSD')

KeyError: 'M_zymst_c'